Train 

In [4]:
import pickle
import numpy as np
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
from keras.callbacks import Callback
from sklearn.preprocessing import LabelEncoder

class F1ScoreCallback(Callback):
    def __init__(self, X_val, y_val):
        super(F1ScoreCallback, self).__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0
        self.best_model = None
        self.f1_scores = []

    def on_epoch_end(self, epoch, logs=None):
        y_val_pred = np.argmax(self.model.predict(self.X_val), axis=1)
        f1 = f1_score(self.y_val, y_val_pred, average='weighted')
        self.f1_scores.append(f1)
        

        if f1 > self.best_f1:
            self.best_f1 = f1
            self.best_model = self.model
            print(f"Epoch {epoch + 1} - F1 Score: {f1:.4f}")
            print("Saved best model")
            print(self.f1_scores)

with open('train_without_test4.pickle', 'rb') as f1:
    balanced = pickle.load(f1)

with open('test_descr_replaced_subj_with_embeddings.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

train = np.array([item['cve_description_replaced_subject_ada_embedding'] for item in balanced if item['cwe'] != 'None'])
test = np.array([item['cwe'] for item in balanced if item['cwe'] != 'None'])
np.random.seed(42)
X_train, X_val, y_train, y_val = train_test_split(train,test,test_size=0.1,random_state=42)

X_test = np.array([item['cve_description_replaced_subject_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

label_encoder_train = LabelEncoder()
y_train_encoded = label_encoder_train.fit_transform(y_train)
label_encoder_test = LabelEncoder()
y_test_encoded = label_encoder_test.fit_transform(y_test)


input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train))

model = Sequential()
model.add(Dense(128, input_dim=input_dim, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

f1_callback = F1ScoreCallback(X_val, label_encoder_train.transform(y_val))

history = model.fit(X_train, y_train_encoded, epochs=60, batch_size=32, validation_data=(X_val, label_encoder_train.transform(y_val)), verbose=1, callbacks=[f1_callback])

best_model = f1_callback.best_model


# Save the best model
joblib.dump(best_model, 'CWE_classes.joblib')

# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_pred_original = label_encoder_train.inverse_transform(y_pred)

print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))

joblib.dump(label_encoder_train, 'label_encoder_train.joblib')

Epoch 1/60
259/259 [==============================] - 1s 2ms/step loss: 2.1536 -
Epoch 1 - F1 Score: 0.4944
Saved best model
[0.4943675509814705]
2323/2323 [==============================] - 10s 4ms/step - loss: 2.1518 - accuracy: 0.4332 - val_loss: 1.6853 - val_accuracy: 0.5358
Epoch 2/60
259/259 [==============================] - 1s 2ms/step loss: 1.557
Epoch 2 - F1 Score: 0.5636
Saved best model
[0.4943675509814705, 0.5635767145664905]
2323/2323 [==============================] - 8s 3ms/step - loss: 1.5563 - accuracy: 0.5654 - val_loss: 1.5153 - val_accuracy: 0.5801
Epoch 3/60
259/259 [==============================] - 0s 2ms/step loss: 1.4315 
Epoch 3 - F1 Score: 0.5682
Saved best model
[0.4943675509814705, 0.5635767145664905, 0.568190693154052]
2323/2323 [==============================] - 9s 4ms/step - loss: 1.4310 - accuracy: 0.5966 - val_loss: 1.4456 - val_accuracy: 0.5914
Epoch 4/60
259/259 [==============================] - 0s 2ms/step loss: 1.3581 
Epoch 4 - F1 Score: 0.6030


['label_encoder_train.joblib']

Run inference

In [1]:
import pickle
import numpy as np
from sklearn.metrics import classification_report
import joblib
from sklearn.preprocessing import LabelEncoder
# Save the best model
with open('test_descr_replaced_subj_with_embeddings.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

X_test = np.array([item['cve_description_replaced_subject_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

best_model=joblib.load('best_model.joblib')
label_encoder_train=joblib.load('label_encoder_train.joblib')
# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_pred_original = label_encoder_train.inverse_transform(y_pred)

print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))

444/444 [==============================] - 1s 2ms/step
Classification Report:
               precision    recall  f1-score   support

         119     0.7550    0.3542    0.4822      1070
         120     0.2665    0.6378    0.3759       196
         125     0.7588    0.7688    0.7638       532
         134     0.6364    0.7368    0.6829        19
         189     0.3152    0.4874    0.3828       119
         190     0.4651    0.7000    0.5589       200
          20     0.4965    0.1765    0.2605       810
         200     0.6638    0.3949    0.4952       590
         203     0.3846    0.7407    0.5063        27
          22     0.8016    0.7568    0.7786       518
         254     0.1379    0.1176    0.1270        34
         255     0.2929    0.4328    0.3494        67
         264     0.5649    0.5368    0.5505       503
         269     0.2647    0.3396    0.2975       106
         276     0.1471    0.3125    0.2000        64
         284     0.2970    0.2439    0.2679       123
  